In [3]:
import os
import uuid
import math
import folium
import random
%matplotlib inline
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn import metrics
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn.metrics import silhouette_score

#### Create dummy data (real data is confidential and cannot be shared)

In [13]:
cities = ['New York', 'Los Angeles', 'Chicago']
states = ['NY', 'CA', 'IL']
city_state_map = {'New York': 'NY', 'Los Angeles': 'CA', 'Chicago': 'IL'}
data = []
for i in range(15):
    city = random.choice(cities)
    state = city_state_map[city]
    lat = round(random.uniform(33.0, 41.0), 6)
    lng = round(random.uniform(-118.0, -87.0), 6)
    store_cnt = 1.0
    street_number = random.randint(1, 9999)
    zip_code = random.randint(10000, 99999)
    address = f"{street_number} {random.choice(['Main St', 'Broadway', 'Elm St', 'Maple Ave', 'Oak St'])}, {city}, {state} {zip_code}"
    data.append([city, state, lat, lng, store_cnt, address])

columns = ['city', 'state', 'lat', 'lng', 'store cnt', 'buyer_shipping_address']
merged_df = pd.DataFrame(data, columns=columns)
print(merged_df)

           city state        lat         lng  store cnt  \
0   Los Angeles    CA  33.641036 -113.966006        1.0   
1      New York    NY  34.621505 -105.118755        1.0   
2      New York    NY  38.049006  -89.426105        1.0   
3      New York    NY  37.502165  -91.975050        1.0   
4       Chicago    IL  35.136653  -92.838052        1.0   
5       Chicago    IL  33.155409  -95.988805        1.0   
6       Chicago    IL  40.019927  -97.896880        1.0   
7   Los Angeles    CA  39.138035 -106.365862        1.0   
8      New York    NY  34.193259  -90.988797        1.0   
9   Los Angeles    CA  35.877519  -98.244203        1.0   
10      Chicago    IL  34.599515 -114.485107        1.0   
11  Los Angeles    CA  37.710336 -109.936879        1.0   
12  Los Angeles    CA  40.797082 -103.776672        1.0   
13      Chicago    IL  37.285052 -110.802880        1.0   
14  Los Angeles    CA  38.360897 -102.172067        1.0   

                  buyer_shipping_address  
0     3943 E

In [14]:
merged_df['id'] = merged_df.reset_index().index + 1
merged_df

,city,state,lat,lng,store cnt,buyer_shipping_address,id
0,Los Angeles,CA,33.641036,-113.966006,1.0,"3943 Elm St, Los Angeles, CA 24685",1
1,New York,NY,34.621505,-105.118755,1.0,"9686 Oak St, New York, NY 57614",2
2,New York,NY,38.049006,-89.426105,1.0,"6222 Broadway, New York, NY 40213",3
3,New York,NY,37.502165,-91.975050,1.0,"9826 Broadway, New York, NY 43687",4
4,Chicago,IL,35.136653,-92.838052,1.0,"8302 Maple Ave, Chicago, IL 74852",5
5,Chicago,IL,33.155409,-95.988805,1.0,"6451 Elm St, Chicago, IL 42017",6
6,Chicago,IL,40.019927,-97.896880,1.0,"3374 Oak St, Chicago, IL 78092",7
7,Los Angeles,CA,39.138035,-106.365862,1.0,"5232 Broadway, Los Angeles, CA 55185",8
8,New York,NY,34.193259,-90.988797,1.0,"7669 Elm St, New York, NY 59568",9
9,Los Angeles,CA,35.877519,-98.244203,1.0,"2414 Broadway, Los Angeles, CA 54324",10


#### input sampling %

In [15]:
data = []
for city, state in zip(cities, states):
    citystate = f"{city}, {state}"
    dbscan_final_pct = 0.0745
    data.append([city, state, citystate, dbscan_final_pct])

columns = ['city', 'state', 'citystate', 'dbscan final %']
city_sampling_data = pd.DataFrame(data, columns=columns)
city_sampling_data = city_sampling_data.drop_duplicates(subset=['city'])
print(city_sampling_data)

          city state        citystate  dbscan final %
0     New York    NY     New York, NY          0.0745
1  Los Angeles    CA  Los Angeles, CA          0.0745
2      Chicago    IL      Chicago, IL          0.0745


In [16]:
df = pd.merge(merged_df, city_sampling_data, on=['city','state'], how='left')
df.rename(columns={'dbscan final %': 'sampling_perc'}, inplace=True)
df['sampling_perc'] = df['sampling_perc'].astype(float)
df['order_cnt'] = 1 / 60 / df['sampling_perc']
df # order_cnt = ado

,city,state,lat,lng,store cnt,buyer_shipping_address,id,citystate,sampling_perc,order_cnt
0,Los Angeles,CA,33.641036,-113.966006,1.0,"3943 Elm St, Los Angeles, CA 24685",1,"Los Angeles, CA",0.0745,0.223714
1,New York,NY,34.621505,-105.118755,1.0,"9686 Oak St, New York, NY 57614",2,"New York, NY",0.0745,0.223714
2,New York,NY,38.049006,-89.426105,1.0,"6222 Broadway, New York, NY 40213",3,"New York, NY",0.0745,0.223714
3,New York,NY,37.502165,-91.975050,1.0,"9826 Broadway, New York, NY 43687",4,"New York, NY",0.0745,0.223714
4,Chicago,IL,35.136653,-92.838052,1.0,"8302 Maple Ave, Chicago, IL 74852",5,"Chicago, IL",0.0745,0.223714
5,Chicago,IL,33.155409,-95.988805,1.0,"6451 Elm St, Chicago, IL 42017",6,"Chicago, IL",0.0745,0.223714
6,Chicago,IL,40.019927,-97.896880,1.0,"3374 Oak St, Chicago, IL 78092",7,"Chicago, IL",0.0745,0.223714
7,Los Angeles,CA,39.138035,-106.365862,1.0,"5232 Broadway, Los Angeles, CA 55185",8,"Los Angeles, CA",0.0745,0.223714
8,New York,NY,34.193259,-90.988797,1.0,"7669 Elm St, New York, NY 59568",9,"New York, NY",0.0745,0.223714
9,Los Angeles,CA,35.877519,-98.244203,1.0,"2414 Broadway, Los Angeles, CA 54324",10,"Los Angeles, CA",0.0745,0.223714


In [17]:
vec = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lat'], df['lng']),crs="EPSG:4326")
print(vec)

           city state        lat         lng  store cnt  \
0   Los Angeles    CA  33.641036 -113.966006        1.0   
1      New York    NY  34.621505 -105.118755        1.0   
2      New York    NY  38.049006  -89.426105        1.0   
3      New York    NY  37.502165  -91.975050        1.0   
4       Chicago    IL  35.136653  -92.838052        1.0   
5       Chicago    IL  33.155409  -95.988805        1.0   
6       Chicago    IL  40.019927  -97.896880        1.0   
7   Los Angeles    CA  39.138035 -106.365862        1.0   
8      New York    NY  34.193259  -90.988797        1.0   
9   Los Angeles    CA  35.877519  -98.244203        1.0   
10      Chicago    IL  34.599515 -114.485107        1.0   
11  Los Angeles    CA  37.710336 -109.936879        1.0   
12  Los Angeles    CA  40.797082 -103.776672        1.0   
13      Chicago    IL  37.285052 -110.802880        1.0   
14  Los Angeles    CA  38.360897 -102.172067        1.0   

                  buyer_shipping_address  id        cit

In [18]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

#### Using DBSCAN to filter outliers

In [23]:
from sklearn.metrics import silhouette_score
rs = pd.DataFrame()
grouped = vec.groupby(['city', 'state']) # state level
locations = []
locations_df = pd.DataFrame()
order = []
order_df = pd.DataFrame()
address_raw_df = pd.DataFrame(columns=["city", "state", "lat", "lng",'buyer_shipping_address'])
count = 0 
for group_name, group_df in grouped:
    print("---------------------------------------------------")
    city_name, state_name = group_name
    print(f"Processing group: {city_name}, {state_name}")
    print('Number of sample points:', len(group_df))
    coords = group_df[['lat', 'lng']].values
    # --------------------------------------------------------------------------------------------------------------
    # Compute DBSCAN
    kilometers = 500
    kms_per_radian = 6371.0088
    epsilon = kilometers / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=2, metric='haversine', algorithm='ball_tree').fit(np.radians(coords))
    # Storing the labels formed
    labels = db.labels_
    # Identifying which points make up the “core points”
    core_points = np.zeros_like(labels, dtype = bool)
    core_points[db.core_sample_indices_] = True
    num_true = core_points.sum()
    print("Number of core points:", num_true)
    # Number of clusters in labels, ignoring noise if present.
    num_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    num_noise_ = list(labels).count(-1)
    print('Number of clusters: {}'.format(num_clusters_))
    # print('Number of core points: {}'.format(num_noise_))
    
    clusters = pd.Series([coords[labels == n] for n in range(num_clusters_)])
    centermost_points = clusters.map(get_centermost_point)
    centermost_points = pd.DataFrame(list(centermost_points), columns=["lat", "lng"])
    rs = pd.concat([centermost_points, rs])
    # --------------------------------------------------------------------------------------------------------------
    # fig, ax = plt.subplots(figsize=[18, 10])
    # # rs_scatter = ax.scatter(centermost_points['lat'], centermost_points['lng'], c='#99cc99', edgecolor='None', alpha=0.7, s=200) # 中心點
    # df_scatter = ax.scatter(group_df['lat'], group_df['lng'], c='k', alpha=0.9, s=3)
    core_coords = group_df[core_points]
    # core_scatter = ax.scatter(core_coords['lat'], core_coords['lng'], c='red', marker='x', label='Core Points')
    percentage_core_points = round((num_true / len(group_df)) * 100, 2)
    # print('Selected core points percentage:', percentage_core_points, '%')  
    # ax.set_title('Full data set vs DBSCAN reduced set')
    # ax.set_xlabel('Latitude')
    # ax.set_ylabel('Longitude')

    # min_lat, max_lat = group_df['lat'].min(), group_df['lat'].max()
    # min_lng, max_lng = group_df['lng'].min(), group_df['lng'].max()
    # ax.set_xlim(min_lat, max_lat)
    # ax.set_ylim(min_lng, max_lng)
    # plt.show()
    # --------------------------------------------------------------------------------------------------------------
    # 將核心點添加到 DataFrame
    address_raw_df = pd.concat([address_raw_df, core_coords], ignore_index=True)
    # --------------------------------------------------------------------------------------------------------------
    # 找出core points>80% hub
    locations.append((city_name, state_name, percentage_core_points))
    temp1 = pd.DataFrame(locations)
    locations_df = pd.concat([locations_df, temp1])
    order.append((city_name, state_name, num_true))
    temp2 = pd.DataFrame(order)
    order_df = pd.concat([order_df, temp2])

---------------------------------------------------
Processing group: Chicago, IL
Number of sample points: 5
Number of core points: 4
Number of clusters: 2
---------------------------------------------------
Processing group: Los Angeles, CA
Number of sample points: 6
Number of core points: 5
Number of clusters: 1
---------------------------------------------------
Processing group: New York, NY
Number of sample points: 4
Number of core points: 3
Number of clusters: 1


#### Summary statistics

In [24]:
order_df = order_df.drop_duplicates()
order_df.columns = ['city', 'state', 'count']
order_df.reset_index(drop=True, inplace=True)
print(order_df)
summary_statistics = order_df.describe()
print(summary_statistics)

          city state  count
0      Chicago    IL      4
1  Los Angeles    CA      5
2     New York    NY      3
       count
count    3.0
mean     4.0
std      1.0
min      3.0
25%      3.5
50%      4.0
75%      4.5
max      5.0


In [25]:
locations_df = locations_df.drop_duplicates()
locations_df.columns = ['city', 'state', 'ratio']
locations_df.reset_index(drop=True, inplace=True)
print(locations_df)
summary_statistics = locations_df.describe()
print(summary_statistics)

          city state  ratio
0      Chicago    IL  80.00
1  Los Angeles    CA  83.33
2     New York    NY  75.00
           ratio
count   3.000000
mean   79.443333
std     4.192807
min    75.000000
25%    77.500000
50%    80.000000
75%    81.665000
max    83.330000


#### Using K-means to cluster order data and find the central location of each cluster as a potential new store location

In [26]:
address_raw_df['city_state'] = address_raw_df['city'] + address_raw_df['state']
address_raw_df['store_cnt'] = address_raw_df['store cnt'].astype(int)
hub_cnt_df = address_raw_df[['city', 'state', 'city_state', 'store_cnt']]
hub_cnt_df = hub_cnt_df.drop_duplicates()
hub_cnt_df

,city,state,city_state,store_cnt
0,Chicago,IL,ChicagoIL,1
4,Los Angeles,CA,Los AngelesCA,1
9,New York,NY,New YorkNY,1


In [27]:
def cleaning_data(order_df):
    data = order_df

    min_longitude = -125.0 
    max_longitude = -66.0
    min_latitude = 25.0
    max_latitude = 49.0

    filtered_data = data[
        (data['lng'] >= min_longitude) & (data['lng'] <= max_longitude) &
        (data['lat'] >= min_latitude) & (data['lat'] <= max_latitude)
    ]

    return filtered_data

In [28]:
def perform_zone_clustering(data, num_clusters):
    X = data[['lng', 'lat']]
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    data['zone_name'] = kmeans.fit_predict(X)    
    
    return data

In [29]:
def zone_clustering(filtered_data, district_df):
    # print(filtered_data)
    district_data = district_df
    result_data = pd.DataFrame(columns=['lng', 'lat', 'city_state', 'zone_name'])
    for index, row in district_data.iterrows():
        district_name = row['city_state']
        
        selected_data = filtered_data[filtered_data['city_state'] == district_name]
        
        # Convert lat and lng columns to numeric types
        selected_data['lat'] = pd.to_numeric(selected_data['lat'])
        selected_data['lng'] = pd.to_numeric(selected_data['lng'])

        # lat_high_threshold = selected_data['lat'].quantile(1)
        # lat_low_threshold = selected_data['lat'].quantile(1)
        # lng_high_threshold = selected_data['lng'].quantile(1)
        # lng_low_threshold = selected_data['lng'].quantile(1)

        # # Filter out outliers
        # selected_data = selected_data[(selected_data['lat'] <= lat_high_threshold) &
        #                               (selected_data['lat'] >= lat_low_threshold) &
        #                               (selected_data['lng'] <= lng_high_threshold) &
        #                               (selected_data['lng'] >= lng_low_threshold)]

        n_samples, _ = selected_data.shape
        n_clusters = row['store_cnt']

        # 進行KMeans分群
        if n_samples >= n_clusters:
            selected_data = perform_zone_clustering(selected_data, n_clusters)
        # else:
        #     selected_data = perform_zone_clustering(selected_data, n_samples)
        
        result_data = pd.concat([result_data, selected_data], ignore_index=True)

    return result_data

In [30]:
def convex_hull_polygon(data):
    data['geometry'] = [Point(x, y) for x, y in zip(data['lng'], data['lat'])]
    gdf = gpd.GeoDataFrame(data, geometry='geometry')
    grouped = gdf.groupby(['city_state', 'zone_name'])
    zone_results = []

    for group_name, group_data in grouped:
        city_state, zone_name = group_name
        
        if not group_data.geometry.is_empty.all():
            geometry = group_data.geometry.unary_union.convex_hull  
            
            if geometry.geom_type == 'Polygon':
                zone_results.append([city_state, zone_name, geometry])
            else:
                print(f"***************Warning {city_state} - {zone_name}The convex hull is not a polygon***************")
        else:
            print(f"***************Error：  {city_state} - {zone_name}Geometry is empty.***************")

    zone_gdf = gpd.GeoDataFrame(zone_results, columns=['city_state','zone_name', 'geometry'], geometry='geometry')


    merged_gdf = pd.merge(zone_gdf, hub_cnt_df[['city','state', 'city_state']], on='city_state', how='left')
    
    return merged_gdf

In [31]:
cleaned_data = cleaning_data(address_raw_df)

In [32]:
kmeans_data = zone_clustering(cleaned_data, hub_cnt_df)
kmeans_data['zone_name'] = kmeans_data['zone_name'] + 1
kmeans_data

,lng,lat,city_state,zone_name,city,state,buyer_shipping_address,store cnt,id,citystate,sampling_perc,order_cnt,geometry,store_cnt


#### Include filtered points

In [33]:
new = pd.merge(df, kmeans_data, on=['id','city','state','citystate','lat','lng','store cnt','buyer_shipping_address','sampling_perc','order_cnt'], how='outer')	
new['zone_name'] = new['zone_name'].astype(float)
new['zone_name'].fillna(0, inplace=True)
print(new)

           city state        lat         lng  store cnt  \
0   Los Angeles    CA  33.641036 -113.966006        1.0   
1      New York    NY  34.621505 -105.118755        1.0   
2      New York    NY  38.049006  -89.426105        1.0   
3      New York    NY  37.502165  -91.975050        1.0   
4       Chicago    IL  35.136653  -92.838052        1.0   
5       Chicago    IL  33.155409  -95.988805        1.0   
6       Chicago    IL  40.019927  -97.896880        1.0   
7   Los Angeles    CA  39.138035 -106.365862        1.0   
8      New York    NY  34.193259  -90.988797        1.0   
9   Los Angeles    CA  35.877519  -98.244203        1.0   
10      Chicago    IL  34.599515 -114.485107        1.0   
11  Los Angeles    CA  37.710336 -109.936879        1.0   
12  Los Angeles    CA  40.797082 -103.776672        1.0   
13      Chicago    IL  37.285052 -110.802880        1.0   
14  Los Angeles    CA  38.360897 -102.172067        1.0   

                  buyer_shipping_address  id        cit

In [34]:
kmeans_data = new.copy()
zone_order_cnt_df = kmeans_data.groupby(['city_state', 'zone_name']).agg(zone_count=('order_cnt', lambda x: x.sum())).reset_index() # ado per zone
district_order_cnt_df = kmeans_data.groupby(['city_state']).agg(ttl_district_count=('order_cnt', lambda x: x.sum())).reset_index() # ado per district
city_order_cnt_df = kmeans_data.groupby(['city']).agg(ttl_city_count=('order_cnt', lambda x: x.sum())).reset_index() # ado per city
address_order_cnt_df = kmeans_data.groupby(['buyer_shipping_address','lng','lat']).agg(ttl_order_count=('order_cnt', lambda x: x.sum())).reset_index() # ado per address
kmeans_data

,city,state,lat,lng,store cnt,buyer_shipping_address,id,citystate,sampling_perc,order_cnt,city_state,zone_name,geometry,store_cnt
0,Los Angeles,CA,33.641036,-113.966006,1.0,"3943 Elm St, Los Angeles, CA 24685",1,"Los Angeles, CA",0.0745,0.223714,NaN,0.0,None,NaN
1,New York,NY,34.621505,-105.118755,1.0,"9686 Oak St, New York, NY 57614",2,"New York, NY",0.0745,0.223714,NaN,0.0,None,NaN
2,New York,NY,38.049006,-89.426105,1.0,"6222 Broadway, New York, NY 40213",3,"New York, NY",0.0745,0.223714,NaN,0.0,None,NaN
3,New York,NY,37.502165,-91.975050,1.0,"9826 Broadway, New York, NY 43687",4,"New York, NY",0.0745,0.223714,NaN,0.0,None,NaN
4,Chicago,IL,35.136653,-92.838052,1.0,"8302 Maple Ave, Chicago, IL 74852",5,"Chicago, IL",0.0745,0.223714,NaN,0.0,None,NaN
5,Chicago,IL,33.155409,-95.988805,1.0,"6451 Elm St, Chicago, IL 42017",6,"Chicago, IL",0.0745,0.223714,NaN,0.0,None,NaN
6,Chicago,IL,40.019927,-97.896880,1.0,"3374 Oak St, Chicago, IL 78092",7,"Chicago, IL",0.0745,0.223714,NaN,0.0,None,NaN
7,Los Angeles,CA,39.138035,-106.365862,1.0,"5232 Broadway, Los Angeles, CA 55185",8,"Los Angeles, CA",0.0745,0.223714,NaN,0.0,None,NaN
8,New York,NY,34.193259,-90.988797,1.0,"7669 Elm St, New York, NY 59568",9,"New York, NY",0.0745,0.223714,NaN,0.0,None,NaN
9,Los Angeles,CA,35.877519,-98.244203,1.0,"2414 Broadway, Los Angeles, CA 54324",10,"Los Angeles, CA",0.0745,0.223714,NaN,0.0,None,NaN


#### hub

In [35]:
polygon_data = convex_hull_polygon(kmeans_data)

In [ ]:
kmeans_directory = r'C:\\Users\\...'
folder_base_name = f'result...'
dir_counter = 1
# Keep trying until a non-existing folder name is found
while True:
    folder_name = folder_base_name if dir_counter == 1 else f'{folder_base_name}_{dir_counter}'
    kmeans_directory = os.path.join(kmeans_directory, folder_name)
    # Check if the folder already exists
    if not os.path.exists(kmeans_directory):
        os.makedirs(kmeans_directory)
        break  # Break the loop once a new directory is created
    else:
        dir_counter += 1

In [ ]:
output_dir = os.path.join(kmeans_directory, 'AllKMeans')
os.makedirs(output_dir, exist_ok=True)
output_all_shapefile = os.path.join(output_dir, 'All.shp')
polygon_data.to_file(output_all_shapefile, encoding='utf-8')

In [36]:
centroid_df = polygon_data.copy()
centroid_df['centroid'] = centroid_df['geometry'].centroid
centroid_df['centroid_lat'] = centroid_df['centroid'].y
centroid_df['centroid_lng'] = centroid_df['centroid'].x

In [ ]:
output_df = pd.merge(centroid_df, zone_order_cnt_df, on=['city_district', 'zone_name'], how='left')
output_df = pd.merge(output_df, district_order_cnt_df, on=['city_district'], how='left')
output_df['perc'] = output_df['zone_count'] / output_df['ttl_district_count']
output_df

#### Order amount covered within a 1.5km radius

In [ ]:
order = kmeans_data.copy()
# order = order[['id','city','district','citydist','lat','lng','buyer_shipping_address','store cnt','order_cnt']]
store = output_df.copy()
store = store[['city_state','centroid_lat','centroid_lng','geometry','centroid','zone_name']]
store['store_id'] = store.reset_index().index + 1

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  
    return c * r

In [ ]:
def assign_store(orders, stores):
    result = []
    for city_district in pd.unique(stores['city_state']):
        district_orders = orders[orders['citystate'] == city_district]
        district_stores = stores[stores['city_state'] == city_district]
        for _, order_row in district_orders.iterrows():
            min_distance = float('inf')
            assigned_district = None
            for _, store_row in district_stores.iterrows():
                distance = haversine(order_row['lng'], order_row['lat'], store_row['centroid_lng'], store_row['centroid_lat'])
                if distance < min_distance:
                    min_distance = distance
                    new_zone_name = store_row['zone_name']
                    assigned_district = store_row['city_state']
                    assigned_store = store_row['store_id']
                    order_id = order_row['id']
                    order_city = order_row['city']
                    order_district = order_row['district']
                    order_lat = order_row['lat']
                    order_lng = order_row['lng']
                    order_addr = order_row['buyer_shipping_address']
                    order_cnt = order_row['order_cnt']
                    store_lat = store_row['centroid_lng']
                    store_lng = store_row['centroid_lat']      
            if min_distance <= 1.5:  
                result.append((min_distance, new_zone_name, assigned_district, assigned_store, order_id, min_distance, order_city, order_district, order_lat, order_lng, order_addr, order_cnt, store_lat, store_lng))
    return pd.DataFrame(result, columns=['min_distance', 'new_zone_name', 'citystate','store_id','order_id', 'distance', 'city', 'state','lat', 'lng', 'addr','order_cnt','store_lat','store_lng'])

assigned_orders = assign_store(order, store)
assigned_orders['is_assigned'] = 1
assigned_orders

In [ ]:
buyer_order = pd.merge(new, assigned_orders, left_on=['city','state','citystate','id','order_cnt','lat','lng'], right_on=['city','state','citystate','order_id','order_cnt','lat','lng'], how='left')	
buyer_order['is_assigned'].fillna(0,inplace=True)
buyer_order

In [ ]:
kmeans_buyer_output_data = buyer_order.copy().drop_duplicates(subset='buyer_shipping_address')
kmeans_buyer_output_data = pd.merge(kmeans_buyer_output_data, district_order_cnt_df, on=['city_state'], how='left')
kmeans_buyer_output_data = pd.merge(kmeans_buyer_output_data, city_order_cnt_df, on=['city'], how='left')
kmeans_buyer_output_data = pd.merge(kmeans_buyer_output_data, address_order_cnt_df, on=['lng','lat','buyer_shipping_address'], how='left')
kmeans_buyer_output_data['district_perc'] = kmeans_buyer_output_data['ttl_order_count'] / kmeans_buyer_output_data['ttl_district_count'] # 該add佔所有district %
kmeans_buyer_output_data['city_perc'] = kmeans_buyer_output_data['ttl_order_count'] / kmeans_buyer_output_data['ttl_city_count']
kmeans_buyer_output_data['city_state_zone'] = kmeans_buyer_output_data['city_state'] + kmeans_buyer_output_data['zone_name'].astype(str)
kmeans_buyer_output_data